In [1]:
import requests
from bs4 import BeautifulSoup
import regex as re
import pandas as pd

Достаем HTML со станциями московского метро 

In [2]:
res = requests.get('https://www.moscowmap.ru/metro.html#lines')
soup = BeautifulSoup(res.text)
html_of_lines = soup.find_all('div', {'class': 'js-metro-stations t-metrostation-list-table'})

Возращает словарик с метками линий

In [3]:
def get_dict_of_lines(soup):
    all_lines = soup.find_all('div', {'class': ['js-toggle-depend', 's-depend-control-single', 's-depend-is-init',
                                                's-depend-control-active']})
    dict_of_lines = dict()

    for i, line in enumerate(all_lines):
        t = line.text.replace(" линия", "")
        t = t.replace(" ", "-")
        dict_of_lines[line.find('span')['class'][-1]] = t

    return dict_of_lines

In [4]:
dict_of_lines = get_dict_of_lines(soup)
print(dict_of_lines)


{'ln-1': 'Сокольническая', 'ln-2': 'Замоскворецкая', 'ln-3': 'Арбатско-Покровская', 'ln-4': 'Филевская', 'ln-5': 'Кольцевая', 'ln-6': 'Калужско-Рижская', 'ln-7': 'Таганско-Краснопресненская', 'ln-8': 'Калининско-Солнцевская', 'ln-9': 'Серпуховско-Тимирязевская', 'ln-10': 'Люблинско-Дмитровская', 'ln-11': 'Большая-Кольцевая-Линия', 'ln-11A': 'Каховская', 'ln-12': 'Бутовская', 'ln-14': 'Центральное-Кольцо', 'ln-15': 'Некрасовская', 'ln-D1': 'МЦД-1', 'ln-D2': 'МЦД-2'}


Название ветки по индексу

In [5]:
def get_line_by_index(index):
    all_lines = soup.find_all('div', {'class': ['js-toggle-depend', 's-depend-control-single', 
                                                's-depend-is-init', 's-depend-control-active']})
    return all_lines[index].find('span')['class'][-1]

In [6]:
# есть ли пересадки у станции
def has_transition(arr_spans):
    return len(arr_spans) > 2

# название станции без кавычек
def get_station(s):
    return re.findall('«(.*?)»', s)[0]

# название ветки по индексу
def get_line_by_index(index):
    all_lines = soup.find_all('div', {'class': ['js-toggle-depend', 's-depend-control-single', 
                                                's-depend-is-init', 's-depend-control-active']})
    return all_lines[index].find('span')['class'][-1]

# название ветки по пересадке
def get_line_by_transition(transition_span):
    return transition_span['class'][1]

Ветка_Станция

In [7]:
def get_full_name(index, station):
    l = dict_of_lines[get_line_by_index(index)].replace(" линия", "")
    b = station.replace(" ", "-")
    return f"{l}_{b}"

Создаем csv со станциями метро

In [8]:
out = pd.DataFrame()
from_stations = []
to_stations = []
w = []
for index, lines in enumerate(html_of_lines):
    all_p = lines.find_all('p')
    prev_station = None
    
    cs = []
    
    l = dict_of_lines[get_line_by_index(index)].lower()
    
    for i, p in enumerate(all_p):
        spans = p.find_all('span')
        station_name = spans[1].text
        
        if "кольц" in l and "большая" not in l and (i == 0 or i == (len(all_p) - 1)):
            cs.append(get_full_name(index, station_name))
            if i == 0:
                w.append(10)
            
        if prev_station is None:
            prev_station = station_name
        else:
            from_stations.append(get_full_name(index, prev_station))
            to_stations.append(get_full_name(index, station_name))
            prev_station = station_name
            w.append(10)
                        
        if has_transition(spans):
            for transition in spans[2:]:
                station_name_to = get_station(transition['title'])
                station_name_to = station_name_to.replace(" ", "-")
                station_name = station_name.replace(" ", "-")
                from_stations.append(f"{dict_of_lines[get_line_by_index(index)]}_{station_name}")
                to_stations.append(f"{dict_of_lines[get_line_by_transition(transition)]}_{station_name_to}")
                w.append(1)
                
    if len(cs) > 0:
        from_stations.append(cs[0])
        to_stations.append(cs[1])
        cs = []
                
out['Start'] = from_stations
out['End'] = to_stations
out['W'] = w

out.to_csv("out.csv", index=False)

out.head()

,Start,End,W
0,Сокольническая_Бульвар-Рокоссовского,Центральное-Кольцо_Бульвар-Рокоссовского,1
1,Сокольническая_Бульвар-Рокоссовского,Сокольническая_Черкизовская,10
2,Сокольническая_Черкизовская,Центральное-Кольцо_Локомотив,1
3,Сокольническая_Черкизовская,Сокольническая_Преображенская-площадь,10
4,Сокольническая_Преображенская-площадь,Сокольническая_Сокольники,10
